# Tutorial: Using the Vortex-Profile Matching Dataset

## Overview

This dataset integrates the **World Ocean Database (WOD)** with the **Mesoscale Eddy Trajectory Atlas (META3.2 DT)**, offering a comprehensive resource to study ocean mesoscale dynamics. It includes over millions of vertical ocean profiles matched with altimetry-tracked eddies data. This tutorial gives an example of access to the dataset through OPeNDAP server.


## We start by importing the necessary packages

In [33]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.lines import Line2D
import gsw
from tqdm import tqdm

We are showing the versions of the packages, but you should be fine if you used the provided `yaml` file to create your `conda`/`mamba` environment.

In [2]:
print("numpy version:", np.__version__)
print("xarray version:", xr.__version__)
print("matplotlib version:", matplotlib.__version__)
print("gsw version:", gsw.__version__)

numpy version: 1.23.5
xarray version: 2024.3.0
matplotlib version: 3.6.2
gsw version: 3.4.0


## Accessing and combining datasets

We here define the bounds of the region of interest. In this case, is the Kuroshio current.

In [4]:
latmin, latmax = 28, 47
lonmin, lonmax = 139, 158

Here we access the dataset and filter profiles within the region of interest. We selected the CTD casts only from 2000 to 2004. It should take a few minutes to load the data. If you need faster access or a larger spatiotemporal domain, you could use `dask` for parallelized access do the server. 

In [5]:
%%time
base_url = "http://www.smast.umassd.edu:8081/thredds/dodsC/Vortex_profiles/vortex_profiles/Matched"
geturl = lambda dataset,variable: f"{base_url}/matched_{dataset.lower()}_{variable.title()}.nc"

dataset = "ctd"
temp = xr.open_dataset(geturl(dataset,"temperature"))
salt = xr.open_dataset(geturl(dataset,"salinity"))

where = lambda ds: (ds.lat>latmin)&(ds.lat<latmax)&(ds.lon>lonmin)&(ds.lon<lonmax)&(ds.time.dt.year>=2000)&(ds.time.dt.year<2005)
temp = temp.isel(casts=np.argwhere(where(temp).values).ravel()).load()
salt = salt.isel(casts=np.argwhere(where(salt).values).ravel()).load()

CPU times: user 1min 26s, sys: 13.5 s, total: 1min 40s
Wall time: 6min 25s


We now combine the temperature and salinity profiles based on common cast ids.

In [6]:
ind = list(set(temp.casts.values).intersection(set(salt.casts.values)))
ds = xr.merge([temp.sel(casts=ind), salt.sel(casts=ind)])

Then estimate the absolute salinity and conservative temperature by utilizing the TEOS-10 Python package (`gsw`) and add them as new variables to the dataset.

In [7]:
p = ds.z
ds["absolute_salinity"] = gsw.SA_from_SP(ds.salinity, p, ds.lon, ds.lat)
ds["conservative_temperature"] = gsw.CT_from_t(ds.absolute_salinity, ds.temperature, p)

To compute anomalies, we need to load data from the World Ocean Atlas and estimate TEOS-10 variables from it.

In [8]:
base_url = "https://www.ncei.noaa.gov/thredds-ocean/dodsC/ncei/woa"
params = ["temperature", "salinity"]

datasets = []
months = np.arange(1, 13)

for param in tqdm(params):
    urls = [f"{base_url}/{param}/decav/1.00/woa18_decav_{param[0]}{m:02.0f}_01.nc" for m in months]
    woa = xr.open_mfdataset(urls, decode_times=False, parallel=True)

    woa = woa.rename(time="month").assign_coords(month=months)  
    datasets.append(woa)

woa = xr.merge(datasets)#.load()
woa = woa.rename(depth="z")
woa = woa.sel(lon=slice(lonmin,lonmax), lat=slice(latmin,latmax))[["s_an", "t_an"]].load()


p = gsw.p_from_z(-woa.z, lat=woa.lat)
woa["clim_absolute_salinity"] = gsw.SA_from_SP(woa.s_an, woa.z, woa.lon, woa.lat)
woa["clim_conservative_temperature"] = gsw.CT_from_t(woa.clim_absolute_salinity, woa.t_an, p)

100%|██████████| 2/2 [00:36<00:00, 18.31s/it]


Then we can estimate the salinity and temperature anomalies within eddies by computing the difference to the climatology interpolated to the location and month of the profiles.

In [9]:
clim = woa.interp(lon=ds.lon, lat=ds.lat, month=ds.time.dt.month, z=ds.z)
ds["anom_absolute_salinity"] = ds.absolute_salinity-clim.clim_absolute_salinity
ds["anom_conservative_temperature"] = ds.conservative_temperature-clim.clim_conservative_temperature

Once we estimate the anomalies, we coul make a plot for all profiles.

In [ ]:
%%time
variables = {
    "anom_conservative_temperature": {
        "label": "Temperature anomaly [$^\circ$C]",
        "xlim": [-12, 12],
    },
    "anom_absolute_salinity": {
        "label": "Salinity anom. [g/kg]",
        "xlim": [-1.3, 1.3],
    },
}

fig, ax = plt.subplots(1,2, figsize=(10,4))

for a, var in zip(ax, variables):
    a.scatter(ds[var][ds.eddy_Ro>0].T, ds.z[ds.eddy_Ro>0].T, s=0.4, c="blue", alpha=0.02);
    a.scatter(ds[var][ds.eddy_Ro<0].T, ds.z[ds.eddy_Ro<0].T, s=0.4, c="red", alpha=0.02);
    
    a.grid(True, linestyle="--", alpha=0.5)
    a.set(ylim=[1000,0], xlabel=variables[var]["label"], xlim=variables[var]["xlim"], ylabel="z [m]")
    a.axvline(0, linestyle="--", color="0.2", zorder=1e3)

# Create custom legend handles
custom_legend = [
    Line2D([0], [0], color="blue", lw=1, label="cyclones", alpha=0.5),
    Line2D([0], [0], color="red", lw=1, label="anticyclones", alpha=0.5),
]

# Add the legend to the first axis
ax[0].legend(handles=custom_legend, loc="lower left")

fig.savefig("../img/anomalies.png", bbox_inches="tight", facecolor="w", dpi=300)

The figure illustrates the vertical profiles of temperature and salinity anomalies for cyclonic (blue) and anticyclonic (red) eddies, extending to 1000 meters. 

Cyclones, characterized by low-pressure systems, typically exhibit negative temperature anomalies near the surface due to the upward displacement of isopycnals (cold cores). These anomalies diminish with increasing depth as the mixing effect of cyclones weakens.

On the other hand, anticyclones, associated with high-pressure systems, show positive temperature anomalies near the surface caused by the downward displacement of isopycnal (warm cores). Similar to cyclones, these anomalies gradually weaken with depth. Notably, beyond 600-800 meters, the influence of both cyclones and anticyclones on the water column diminishes, and the anomalies converge around zero.

The salinity anomalies are more complex since they depend on the distribution of water masses in the region.

This plot effectively captures the contrasting thermal structures of cyclones and anticyclones, highlighting their impact on ocean stratification, heat transport, and mixing processes. Additionally, the visualization serves as a validation of the dataset by demonstrating consistency with theoretical expectations of mesoscale eddy behavior..
